In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

from pprint import pprint  # pretty-printer
from collections import defaultdict

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [5]:
from gensim import corpora
dictionary = corpora.Dictionary(texts)
dictionary.save('./tmp/deerwester.dict')  # store the dictionary, for future reference
print(dictionary)

2023-07-04 13:37:49,422 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2023-07-04 13:37:49,424 : INFO : built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)
2023-07-04 13:37:49,427 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)", 'datetime': '2023-07-04T13:37:49.427280', 'gensim': '4.3.1', 'python': '3.8.2 (default, Mar 26 2020, 10:43:30) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'created'}
2023-07-04 13:37:49,429 : INFO : Dictionary lifecycle event {'fname_or_handle': './tmp/deerwester.dict', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-07-04T13:37:49.429519', 'gensim': '4.3.1', 'python': '3.8.2 (default, Mar 26 2020, 10:43:30) \n[Clang 4.0.1 (tags/REL

Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...>


In [7]:
dictionary.token2id

{'computer': 0,
 'human': 1,
 'interface': 2,
 'response': 3,
 'survey': 4,
 'system': 5,
 'time': 6,
 'user': 7,
 'eps': 8,
 'trees': 9,
 'graph': 10,
 'minors': 11}

In [8]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)  # the word "interaction" does not appear in the dictionary and is ignored

[(0, 1), (1, 1)]


In [9]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('./tmp/deerwester.mm', corpus)  # store to disk, for later use
print(corpus)

2023-07-04 13:49:02,858 : INFO : storing corpus in Matrix Market format to ./tmp/deerwester.mm
2023-07-04 13:49:02,860 : INFO : saving sparse matrix to ./tmp/deerwester.mm
2023-07-04 13:49:02,861 : INFO : PROGRESS: saving document #0
2023-07-04 13:49:02,861 : INFO : saved 9x12 matrix, density=25.926% (28/108)
2023-07-04 13:49:02,862 : INFO : saving MmCorpus index to ./tmp/deerwester.mm.index


[[(0, 1), (1, 1), (2, 1)], [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(2, 1), (5, 1), (7, 1), (8, 1)], [(1, 1), (5, 2), (8, 1)], [(3, 1), (6, 1), (7, 1)], [(9, 1)], [(9, 1), (10, 1)], [(9, 1), (10, 1), (11, 1)], [(4, 1), (10, 1), (11, 1)]]


# Corpus Streaming – One Document at a Time

In [10]:
from smart_open import open  # for transparently opening remote files


class MyCorpus:
    def __iter__(self):
        for line in open('https://radimrehurek.com/mycorpus.txt'):
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(line.lower().split())

In [11]:
corpus_memory_friendly = MyCorpus()  # doesn't load the corpus into memory!
print(corpus_memory_friendly)

In [12]:
for vector in corpus_memory_friendly:  # load one vector into memory at a time
    print(vector)

[(0, 1), (1, 1), (2, 1)]
[(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(2, 1), (5, 1), (7, 1), (8, 1)]
[(1, 1), (5, 2), (8, 1)]
[(3, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(4, 1), (10, 1), (11, 1)]


## # construct the dictionary without loading all texts into memory:

In [13]:
# collect statistics about all tokens
dictionary = corpora.Dictionary(line.lower().split() for line in open('https://radimrehurek.com/mycorpus.txt'))
# remove stop words and words that appear only once
stop_ids = [
    dictionary.token2id[stopword]
    for stopword in stoplist
    if stopword in dictionary.token2id
]
once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.items() if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids)  # remove stop words and words that appear only once
dictionary.compactify()  # remove gaps in id sequence after words that were removed
print(dictionary)

KeyboardInterrupt: 

In [14]:
corpus = [[(1, 0.5)], []]  # make one document empty, for the heck of it

corpora.MmCorpus.serialize('./tmp/corpus.mm', corpus)

2023-07-04 14:47:44,918 : INFO : storing corpus in Matrix Market format to ./tmp/corpus.mm
2023-07-04 14:47:44,920 : INFO : saving sparse matrix to ./tmp/corpus.mm
2023-07-04 14:47:44,920 : INFO : PROGRESS: saving document #0
2023-07-04 14:47:44,921 : INFO : saved 2x2 matrix, density=25.000% (1/4)
2023-07-04 14:47:44,922 : INFO : saving MmCorpus index to ./tmp/corpus.mm.index


In [16]:
corpus = corpora.MmCorpus('./tmp/corpus.mm')
print(corpus)

2023-07-04 14:48:19,260 : INFO : loaded corpus index from ./tmp/corpus.mm.index
2023-07-04 14:48:19,261 : INFO : initializing cython corpus reader from ./tmp/corpus.mm
2023-07-04 14:48:19,262 : INFO : accepted corpus with 2 documents, 2 features, 1 non-zero entries


MmCorpus(2 documents, 2 features, 1 non-zero entries)


In [17]:
list(corpus)

[[(1, 0.5)], []]

In [18]:
# another way of doing it: print one document at a time, making use of the streaming interface
for doc in corpus:
    print(doc)

[(1, 0.5)]
[]
